In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import sys
import os 

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

import warnings

sys.path.append(os.path.abspath("../"))

from src.Components.Data_ingestion import TypeFile

In [3]:
from src.Components.Data_transformation import traintestsplit

In [4]:
file_path = "/home/amine/Desktop/Projects/endtoend_DS_withdeploy/Notebook/Data/stud.csv"

In [5]:
df = TypeFile().Ingest(file_path)

In [6]:
df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [7]:
x_train, x_test, y_train, y_test = traintestsplit().DataTra(file_path)

In [8]:
x_train.shape , x_test.shape , y_train.shape , y_test.shape

((750, 19), (250, 19), (750,), (250,))

In [9]:
Models = {
    
    "Decision Tree": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "Lasso": Lasso(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "LinearRegression": LinearRegression(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoostRegressor": CatBoostRegressor(),
    "Ridge": Ridge(),
    "svm": SVR()
    
}

In [10]:
from src.Components.Model_Training import FitModel

In [11]:
model, r2_score = FitModel().Compute(x_train, x_test, y_train, y_test, Models)


 Model Decision Tree ==> 

 Model Performance on Trainig set

 r square: 0.9999885726649873, mean absolute error: 0.0026666666666666666, mean square error: 0.0026666666666666666

 Model Performance on Testing set

 r square: 0.7186524452547992, mean absolute error: 6.26, mean square error: 61.308



 Model RandomForestRegressor ==> 

 Model Performance on Trainig set

 r square: 0.9771800502465253, mean absolute error: 1.818565777777778, mean square error: 5.32523106

 Model Performance on Testing set

 r square: 0.8335728230891275, mean absolute error: 4.88002, mean square error: 36.2658825

 Model Lasso ==> 

 Model Performance on Trainig set

 r square: 0.8133535448734424, mean absolute error: 5.21231229306112, mean square error: 43.555551647413225

 Model Performance on Testing set

 r square: 0.7897578230919235, mean absolute error: 5.3501729572090335, mean square error: 45.81353974643073

 Model AdaBoostRegressor ==> 

 Model Performance on Trainig set

 r square: 0.8528222021373233, mean absolute error: 4.763209420383995, mean square error: 34.345201851348904

 Model Performance on Testing set

 r square: 0.801430843917647, mean absolute error: 5.244558731924571, mean square error: 43.26989026836231

 Model LinearRegression ==> 

 Model Performance on Trainig set

 r squar

In [12]:
pd.DataFrame(list(zip(Models, r2_score)), columns=['Model', 'R2 Square']).sort_values(by=['R2 Square'], ascending=False)

,Model,R2 Square
8,Ridge,0.852477
4,LinearRegression,0.852204
7,CatBoostRegressor,0.837211
1,RandomForestRegressor,0.833573
3,AdaBoostRegressor,0.801431
6,XGBRegressor,0.801200
2,Lasso,0.789758
5,KNeighborsRegressor,0.775863
9,svm,0.770577
0,Decision Tree,0.718652


In [13]:
model = Ridge()

model.fit(x_train, y_train)
y_p = model.predict(x_test)

In [14]:
pd = pd.DataFrame({'Actual value':y_test, 'Predicted value':y_p, 'Deffirence':y_test - y_p})

In [15]:
pd

,Actual value,Predicted value,Deffirence
993,62,62.368880,-0.368880
859,87,75.231574,11.768426
298,40,48.592588,-8.592588
553,77,66.383568,10.616432
672,69,69.646723,-0.646723
...,...,...,...
462,71,72.663662,-1.663662
356,63,66.887122,-3.887122
2,90,84.729805,5.270195
478,55,58.912522,-3.912522
